<a href="https://colab.research.google.com/github/alanwuha/ce7455-nlp/blob/master/practice/SkipGram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skip Gram
- [Ref](https://blog.cambridgespark.com/tutorial-build-your-own-embedding-and-use-it-in-a-neural-network-e9cde4a81296?gi=1cb19fad9b7a)
- [Code](https://github.com/DSKSD/DeepNLP-models-Pytorch/blob/master/notebooks/02.Skip-gram-Negative-Sampling.ipynb)
- Edited by Hancheol Moon

![skip-gram](https://cdn-images-1.medium.com/max/800/1*SR6l59udY05_bUICAjb6-w.png)

- Skip-gram's objective is to predict the contexts, given a target word: V_t -> V_c
- The contexts are immediate neighbours of the target and are retrieved using a window of an arbitrary size _n_
  - Capturing _n_ words to the left of the target and _n_ words to its right.
- In a two-gram example:

![two-gram](https://nbviewer.jupyter.org/github/DSKSD/DeepNLP-models-Pytorch/blob/master/images/01.skipgram-prepare-data.png)

- The original Skip-grams' objective is to maximize P(V_c|V_c): The probability of V_c being predicted as V_t's context for all training pairs.
- To calculate P(V_c|V_t) we need a way to quantify the __closeness__ of the target-word around the context-word.
- In Skip-gram, this closeness is computed using the __dot product between the input-embedding of the target and the output-embedding of the context__.

Now, if we define u_t,c to be the measure of words' closeness between the target word and context word, _E_ to be the embedding matrix holding input-embeddings and _O_ to be the output-embedding matrix we get:

u_t,c = E_t O_c

, where _c_ is the context and _t_ is the target. With softmax,

![architecture](https://cdn-images-1.medium.com/max/1600/1*4Viy_LvP6jLIWSvB9-Fk-Q.png)

# Negative Sampling

So far, we have studied the basics of Skip-gram, but there is an issue with the __original softmax objective of Skip-gram__. It is __highly computationally expensive__:
- It requires scanning through the output-embeddings of all words in the vocabulary in order to calculate the sum from the __denominator__.
- Typically such vocabularies contain hundreds of thousands of words.
Because of this inefficiency, most implementations use an alternative, _negative-sampling objective_, which rephrases the problem as a set of independent binary classification tasks.

Instead of defining the complete probability distribution over words, __the model learns to distinguish the correct training pairs from incorrect pairs, which are randomly generated pairs.__
- Negative pair: Keep V_t and sample V_c from noise distribution
- D: correct pairs
- D': all negatively sampled |D| x m pairs
- P(C = 1 | V_t, V_c): the probability of (V_t, V_t) being a correct pair

For each sample we are making a binary decision we define P(C = 1|V_t, V_c) using the sigmoid function.

Negative (context) samples are drawn from uniform distribution raised to the power of 3/4. Why? If you play with some sample values, you'll find that, compared to the simpler equation, this one has the tendency to increase the probability for less frequent words and decrease the probability for more frequent words.

P(w) = Unif(W)^3/4 / Z,

where Z is the normalization factor.

__Sampling-based approaches completely do away with the softmax layer.__ They do this by approximating the normalization in the denominator of the softmax with some other loss that is cheap to compute. __However, sampling-based approaches are only useful at training time - - during inference, the full softmax still needs to be computed to obtain a normalised probability.

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional

import nltk
import random
import numpy as np
from collections import Counter

import pdb

flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(1024)

In [0]:
def getBatch(batch_size, train_data):
  random.shuffle(train_data) # Shuffling is necessary. Why?
  sindex = 0
  eindex = batch_size
  while eindex < len(train_data):
    batch = train_data[sindex:eindex]
    temp = eindex
    eindex = eindex + batch_size
    sindex = temp
    yield batch

  if eindex >= len(train_data):
    batch = train_data[sindex:]
    yield batch

In [0]:
def prepare_sequence(seq, word2index):
  l = lambda w: word2index[w] if word2index.get(w) is not None else word2index['<UNK>']
  idxs = list(map(l, seq))
  return torch.Tensor(idxs).type(torch.LongTensor)

def prepare_word(word, word2index):
  return torch.Tensor([word2index[word]]).type(torch.LongTensor) if word2index.get(word) is not None else torch.LongTensor([word2index['<UNK>']])

In [32]:
nltk.download('punkt')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [33]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [0]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:500] # sampling sentences for test
corpus = [[word.lower() for word in sent] for sent in corpus]

In [35]:
print(len(corpus))
print(corpus[0], len(corpus[0]))

500
['[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']'] 8


In [0]:
# Exclude sparse words
MIN_COUNT = 3
word_count = Counter(flatten(corpus))
exclude = []
for w, c in word_count.items():
  if c < MIN_COUNT:
    exclude.append(w)

In [52]:
vocab = list(set(flatten(corpus)) - set(exclude))
vocab.append('<UNK>')
len(vocab)

479

In [0]:
word2index = {'<UNK>': 0}

for vo in vocab:
  if word2index.get(vo) is None:
    word2index[vo] = len(word2index)

  index2word = {v:k for k, v in word2index.items()}

In [0]:
WINDOW_SIZE = 5 # Range of contexts
windows = flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE,
                                    WINDOW_SIZE * 2 + 1)) for c in corpus])

In [68]:
print(windows[0])
print(windows[1])

('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville')
('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville', '1851')


In [65]:
list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + corpus[0] + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1))

[('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville'),
 ('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851'),
 ('<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']'),
 ('<DUMMY>',
  '<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>'),
 ('<DUMMY>',
  '[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>'),
 ('[',
  'moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>'),
 ('moby',
  'dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>'),
 ('dick',
  'by',
  'herman',
  'melville',
  '1851',
  ']',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>',
  '<DUMMY>')]

In [0]:
# Create Training Set
train_data = []
for window in windows:
  for i in range(WINDOW_SIZE * 2 + 1):
    if window[i] in exclude or window[WINDOW_SIZE] in exclude:
      continue  # min_count

    if i == WINDOW_SIZE or window[i] == '<DUMMY>':
      continue

    # window[WINDOW_SIZE] : target word
    # window[i]           : context word

    train_data.append((window[WINDOW_SIZE], window[i]))

In [70]:
# 2-Gram dataset
train_data[:10]

[('(', 'supplied'),
 ('(', 'by'),
 ('(', 'a'),
 ('(', 'late'),
 ('supplied', '('),
 ('supplied', 'by'),
 ('supplied', 'a'),
 ('supplied', 'late'),
 ('by', '('),
 ('by', 'supplied')]

In [0]:
X_p = []
y_p = []

for tr in train_data:
  X_p.append(prepare_word(tr[0], word2index).view(1, -1))
  y_p.append(prepare_word(tr[1], word2index).view(1, -1))

train_data = list(zip(X_p, y_p))